# Scrape Combined ND Football Schedules From Sports Reference



# Load Modules Using Import

In [ ]:
# import requests
import requests

# import csv
import csv

# import beautifulsoup
from bs4 import BeautifulSoup
from bs4 import Comment

# import pandas
import pandas as pd

# import numpy
import numpy as np

# Get List of Season URLs from Sports Reference

In [ ]:
# root url
root = "https://www.sports-reference.com/cfb/schools/notre-dame/"

# year range
years = range(1899, 2021, 1)

# url tag
tag = "-schedule.html"

# empty list for urls
urls = []

# for loop that concatenates full url
for year in years:
    urls.append(root + str(year) + tag)
    
# show list of urls
urls

# Create Dictionary That Connects Season Year With URL

In [ ]:
test_dict = {years[i]: urls[i] for i in range(len(years))}

test_dict

# Create Dictionary of Pandas DataFrames with Each Year's Schedule

NOTE: The 1899-1935 seasons do not include an AP poll summary table  before the game information and results. The `if-else` statement in the `for` loop accounts for this varying page table structure.

In [ ]:
# empty dictionary
schedules = {}

# for loop that loads schedule as pandas data frame and saves dataframe as dictionary value associated with season year key
for key, value in test_dict.items():
    df_name = key
    if key >= 1899 | key <= 1935:
        try:
            dfs = pd.read_html(value)
            schedule = dfs[0]
            schedule.loc[:, 'Season'] = key
            schedules[df_name] = schedule
        except:
            continue
    else:
        try:
            dfs = pd.read_html(value)
            schedule = dfs[1]
            schedule.loc[:, 'Season'] = key
            schedules[df_name] = schedule
        except:
            continue

# Combine Dictionary of DataFrames using `pd.concat`

In [ ]:
# create combined_schedule df
combined_schedule = pd.concat(schedules.values(), ignore_index=True)

# show df
combined_schedule

# Data Cleaning/Wrangling

In [ ]:
# rename columns
combined_schedule.rename(columns = {'Unnamed: 4': 'Location', 'Unnamed: 7': 'Result'}, inplace= True)

# show updated dataframe
combined_schedule

In [ ]:
# take non-null values from second Location column ('Unnamed: 5') and assign to cells in 'Location' column
combined_schedule["Location"] = np.where(combined_schedule["Unnamed: 5"].notnull(), combined_schedule["Unnamed: 5"], combined_schedule["Location"])

# take non-null values from second Result column ('Unnamed: 8') and assign to cells in 'Result' column
combined_schedule["Result"] = np.where(combined_schedule["Unnamed: 8"].notnull(), combined_schedule["Unnamed: 8"], combined_schedule["Result"])

# show updated dataframe
combined_schedule

In [ ]:
# remove second location column
del combined_schedule['Unnamed: 5']

# remove second result column
del combined_schedule['Unnamed: 8']

# show updated dataframe
combined_schedule

In [ ]:
# replace @ symbols in Location column with 'Away' string
combined_schedule['Location'] = combined_schedule['Location'].str.replace("@", "Away")

# replace 'N' string in Location column with 'Neutral' string
combined_schedule['Location'] = combined_schedule['Location'].str.replace("N", "Neutral")

# replace NaN values in Location column with 'Home' string
combined_schedule['Location'] = combined_schedule['Location'].fillna("Home")

# show updated dataframe
combined_schedule

In [ ]:
# create duplicate School column to standardize school name
combined_schedule['Standardized_School'] = combined_schedule['School']

# remove rankings from new column
combined_schedule['Standardized_School']= combined_schedule['Standardized_School'].str.replace(r'\d', '')
combined_schedule['Standardized_School']= combined_schedule['Standardized_School'].str.replace(r'[()]', '')

# show updated dataframe
combined_schedule

In [ ]:
# create duplicate Opponents column to standardize opponent name
combined_schedule['Standardized_Opponent'] = combined_schedule['Opponent']

# remove rankings from new column
combined_schedule['Standardized_Opponent']= combined_schedule['Standardized_Opponent'].str.replace(r'\d', '')
combined_schedule['Standardized_Opponent']= combined_schedule['Standardized_Opponent'].str.replace(r'[()]', '')

# show updated dataframe
combined_schedule

In [ ]:
# create new column for converted datetime object
combined_schedule['Standardized_Date'] = pd.to_datetime(combined_schedule['Date'])

combined_schedule

In [ ]:
combined_schedule.info()

In [ ]:
# list with desired column order
column_names = ['G', 'Season', 'Standardized_Date', 'Date', 'Day', 'Time', 'School', 'Standardized_School', 'Location', 'Opponent', 'Standardized_Opponent', 'Conf', 'Result', 'Pts', 'Opp', 'W', 'L', 'T', 'Streak', 'Notes']

# reorder columns
combined_schedule = combined_schedule.reindex(columns = column_names)

# show updated dataframe
combined_schedule

# Write DataFrame to CSV File

In [ ]:
# write combined_schedules dataframe to csv file
combined_schedule.to_csv('combined_nd_schedules.csv', index=False)